# Colab 데이터 로드

In [ ]:
# Step 1.분석할 데이터가 저장된 파일을 불러와서 변수에 할당합니다.
from google.colab import files
myfile = files.upload()
import io
import pandas as pd
#pd.read_csv로 csv파일 불러오기
src_data = pd.read_csv(io.BytesIO(myfile['와인.csv']),
                       encoding='cp949')
src_data

Saving 와인.csv to 와인.csv


,알콜도수,당도,산도_ph,종류
0,9.4,1.9,3.51,레드와인
1,9.8,2.6,3.20,레드와인
2,9.8,2.3,3.26,레드와인
3,9.8,1.9,3.16,레드와인
4,9.4,1.9,3.51,레드와인
...,...,...,...,...
6492,11.2,1.6,3.27,화이트와인
6493,9.6,8.0,3.15,화이트와인
6494,9.4,1.2,2.99,화이트와인
6495,12.8,1.1,3.34,화이트와인


# 로컬 데이터 로드

In [ ]:
import pandas as pd
src_data = pd.read_csv('와인.csv', encoding='cp949')
src_data

,알콜도수,당도,산도_ph,종류
0,9.4,1.9,3.51,레드와인
1,9.8,2.6,3.20,레드와인
2,9.8,2.3,3.26,레드와인
3,9.8,1.9,3.16,레드와인
4,9.4,1.9,3.51,레드와인
...,...,...,...,...
6492,11.2,1.6,3.27,화이트와인
6493,9.6,8.0,3.15,화이트와인
6494,9.4,1.2,2.99,화이트와인
6495,12.8,1.1,3.34,화이트와인


# 공통 실습 코드

In [ ]:
from sklearn.model_selection import train_test_split
# Step 2.주어진 데이터를 훈련용 데이터와 검증용 데이터로 나눕니다
import numpy as np

# '알콜도수','당도','산도_ph'가지고 와인 종류 분류하기
data =  src_data[['알콜도수','당도','산도_ph']]
target = src_data['종류']
print(data)
print(target)

# tarin test 데이터 분리
훈련용_data, 테스트용_data, 훈련용_target, 테스트용_target = train_test_split(data, target, test_size=0.2, random_state=10)

      알콜도수   당도  산도_ph
0      9.4  1.9   3.51
1      9.8  2.6   3.20
2      9.8  2.3   3.26
3      9.8  1.9   3.16
4      9.4  1.9   3.51
...    ...  ...    ...
6492  11.2  1.6   3.27
6493   9.6  8.0   3.15
6494   9.4  1.2   2.99
6495  12.8  1.1   3.34
6496  11.8  0.8   3.26

[6497 rows x 3 columns]
0        레드와인
1        레드와인
2        레드와인
3        레드와인
4        레드와인
        ...  
6492    화이트와인
6493    화이트와인
6494    화이트와인
6495    화이트와인
6496    화이트와인
Name: 종류, Length: 6497, dtype: object


In [ ]:
# 교차검증 없이 모델 검증합니다.
from sklearn.tree import DecisionTreeClassifier
# 모델 생성
df = DecisionTreeClassifier(random_state=10)
# 모델 학습
df.fit(훈련용_data, 훈련용_target)
# 모델 score 출력
print(df.score(훈련용_data, 훈련용_target))
print(df.score(테스트용_data, 테스트용_target))

0.9974985568597268
0.8515384615384616


In [ ]:
# Step 3.K-Fold 교차 검증 수행(K값 자율 설정)
# cross_validate , cross_val_score 출력
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.tree import DecisionTreeClassifier
df = DecisionTreeClassifier(random_state=10)
scores_1=cross_validate(df, 훈련용_data, 훈련용_target, cv=5)
scores_2=cross_val_score(df, 훈련용_data, 훈련용_target, cv=5)

print("cross_validate 결과:",scores_1)
print("cross_validate 결과:",np.mean(scores_1['test_score']))

print("cross_val_score 결과:",scores_2)
print("cross_val_score 결과:",np.mean(scores_2))

cross_validate 결과: {'fit_time': array([0.01541758, 0.01343131, 0.01334381, 0.0144453 , 0.02021027]), 'score_time': array([0.00408411, 0.00363111, 0.00371695, 0.00622559, 0.0063858 ]), 'test_score': array([0.8625    , 0.84326923, 0.85274302, 0.86044273, 0.87006737])}
cross_validate 결과: 0.8578044717553862
cross_val_score 결과: [0.8625     0.84326923 0.85274302 0.86044273 0.87006737]
cross_val_score 결과: 0.8578044717553862


In [ ]:
#그리드서치를 사용하여 하이퍼 파라미터 튜닝하기
from sklearn.model_selection import GridSearchCV
parm = {'max_depth': range(1,10,1),
        'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
        'min_samples_split':range(2,20,1)}

gs = GridSearchCV(DecisionTreeClassifier(random_state=50), parm, n_jobs=-1)
gs.fit(훈련용_data, 훈련용_target)
print(gs.best_params_)

{'max_depth': 9, 'min_impurity_decrease': np.float64(0.0005), 'min_samples_split': 2}


In [ ]:
# 훈련용, 테스트용 데이터에 대해 최적의 결과값 출력
print(gs.best_params_)
df = gs.best_estimator_
print(df.score(훈련용_data, 훈련용_target))
print(df.score(테스트용_data, 테스트용_target))

{'max_depth': 9, 'min_impurity_decrease': np.float64(0.0005), 'min_samples_split': 2}
0.8916682701558591
0.8653846153846154
